In [10]:
from pathlib import Path
import os, glob, traceback
import torch
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor

NNUNET_ROOT = "/workspace/nnunet_seg_model"
MODEL_DIR   = str(Path(NNUNET_ROOT) / "models" / "Dataset505_PelvicBone" / "nnUNetTrainer__nnUNetPlans__3d_fullres")

IN_DIR  = "/workspace/MRI/output"
OUT_DIR = "/workspace/MRI/seg_out"
os.makedirs(OUT_DIR, exist_ok=True)

# CUDA 사용 가능 여부 점검
if not torch.cuda.is_available():
    raise SystemExit("CUDA GPU를 찾지 못했습니다. NVIDIA GPU + CUDA 휠(torch) 설치를 확인하세요.")

cases = sorted(glob.glob(os.path.join(IN_DIR, "*_0000.nii.gz")))
assert cases, f"No '*_0000.nii.gz' in {IN_DIR}"
file_lists = [[c] for c in cases]
print("Cases:", len(cases))

dev = torch.device('cuda')  # ✅ GPU 사용
# 정확도 모드(느리면 아래 Fast 모드 참고)
predictor = nnUNetPredictor(
    tile_step_size=0.5,      # 겹침 넉넉 (정확도↑)
    use_gaussian=True,       # 블렌딩 On
    use_mirroring=True,      # TTA On (정확도↑, 속도↓)
    device=dev,
    verbose=True,
    verbose_preprocessing=True,
)

print("Init from:", MODEL_DIR)
predictor.initialize_from_trained_model_folder(
    model_training_output_dir=MODEL_DIR,
    checkpoint_name="checkpoint_best.pth",
    use_folds=(0,1,2),       # 3-fold 앙상블 (정확도↑, 속도↓)
)

try:
    predictor.predict_from_files(
        file_lists, OUT_DIR,
        save_probabilities=False,
        overwrite=True,
        num_processes_preprocessing=2,     # GPU I/O 보조
        num_processes_segmentation_export=2,
    )
    print("✅ Done ->", OUT_DIR)
except Exception:
    traceback.print_exc()


Cases: 54
Init from: /workspace/nnunet_seg_model/models/Dataset505_PelvicBone/nnUNetTrainer__nnUNetPlans__3d_fullres
There are 54 cases in the source folder
I am processing 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 54 cases that I would like to predict
nnUNet_raw is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
nnUNet_results is not defined and nnU-Net cannot be used for training or inference. If this is not intended behavior, please read documentation/setting_up_paths.md for information on how to se

  0%|          | 0/168 [00:00<?, ?it/s]

old shape: (34, 500, 501), new_shape: [  51 1050 1052], old_spacing: [np.float64(5.9999871253967285), np.float64(0.8203020095825195), np.float64(0.8203099966049194)], new_spacing: [3.9999987840652467, 0.3905999958515167, 0.3905999958515167], fn_data: functools.partial(<function resample_data_or_seg_to_shape at 0x7b1d57772a70>, is_seg=False, order=3, order_z=0, force_separate_z=None)
None


 15%|█▌        | 26/168 [00:10<00:51,  2.78it/s]

old shape: (35, 504, 512), new_shape: [ 52 832 845], old_spacing: [np.float64(5.999983310699463), np.float64(0.6445000171661377), np.float64(0.6445000171661377)], new_spacing: [3.9999987840652467, 0.3905999958515167, 0.3905999958515167], fn_data: functools.partial(<function resample_data_or_seg_to_shape at 0x7dca2b1daa70>, is_seg=False, order=3, order_z=0, force_separate_z=None)


...

Input shape: torch.Size([1, 62, 699, 1072])
step_size: 0.5
mirror_axes: (0, 1, 2)
n_steps 192, image size is torch.Size([62, 699, 1072]), tile_size [20, 320, 256], tile_step_size 0.5
steps:
[[0, 8, 17, 25, 34, 42], [0, 126, 253, 379], [0, 117, 233, 350, 466, 583, 699, 816]]
move image to device cuda
preallocating results arrays on device cuda


100%|██████████| 192/192 [01:43<00:00,  1.86it/s]


Input shape: torch.Size([1, 62, 699, 1072])
step_size: 0.5
mirror_axes: (0, 1, 2)
n_steps 192, image size is torch.Size([62, 699, 1072]), tile_size [20, 320, 256], tile_step_size 0.5
steps:
[[0, 8, 17, 25, 34, 42], [0, 126, 253, 379], [0, 117, 233, 350, 466, 583, 699, 816]]
move image to device cuda
preallocating results arrays on device cuda


100%|██████████| 192/192 [01:41<00:00,  1.89it/s]


Prediction done
sending off prediction to background worker for resampling and export
done with TCGA-VS-AA62
✅ Done -> /MRI/seg_out


In [11]:
from pathlib import Path
import os, glob, traceback
import torch
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor

NNUNET_ROOT = "/workspace/nnunet_seg_model"
MODEL_DIR   = str(Path(NNUNET_ROOT) / "models" / "Dataset505_PelvicBone" / "nnUNetTrainer__nnUNetPlans__3d_fullres")

IN_DIR  = "/workspace/MRI/output"
OUT_DIR = "/MRI/seg_out"
os.makedirs(OUT_DIR, exist_ok=True)

# 필터 키워드(파일명에 이 문자열이 포함된 것만 남김)
INCLUDE_KEYWORDS = ["CCTH"]     # 필요 시 ["CCTH", "MR1"] 처럼 추가 가능
EXCLUDE_KEYWORDS = ["TCGA"]   # 필요하면 제외 키워드도 사용

# CUDA 체크
if not torch.cuda.is_available():
    raise SystemExit("CUDA GPU를 찾지 못했습니다. NVIDIA GPU + CUDA 휠(torch) 설치를 확인하세요.")

# 모든 *_0000.nii.gz 수집
cases_all = sorted(glob.glob(os.path.join(IN_DIR, "*_0000.nii.gz")))
assert cases_all, f"No '*_0000.nii.gz' in {IN_DIR}"

# CCTH만 필터
def keep(path):
    name = os.path.basename(path).upper()
    if not any(k.upper() in name for k in INCLUDE_KEYWORDS):
        return False
    # if any(k.upper() in name for k in EXCLUDE_KEYWORDS):  # 제외가 필요하면 주석 해제
    #     return False
    return True

cases = [c for c in cases_all if keep(c)]
assert cases, f"No CCTH '*_0000.nii.gz' in {IN_DIR}. (Total found: {len(cases_all)})"

file_lists = [[c] for c in cases]
print(f"Cases (CCTH-only): {len(cases)} / Total: {len(cases_all)}")
print("Sample:", [os.path.basename(c) for c in cases[:5]])

dev = torch.device('cuda')
predictor = nnUNetPredictor(
    tile_step_size=0.5,
    use_gaussian=True,
    use_mirroring=True,
    device=dev,
    verbose=True,
    verbose_preprocessing=True,
)

print("Init from:", MODEL_DIR)
predictor.initialize_from_trained_model_folder(
    model_training_output_dir=MODEL_DIR,
    checkpoint_name="checkpoint_best.pth",
    use_folds=(0,1,2),
)

try:
    predictor.predict_from_files(
        file_lists, OUT_DIR,
        save_probabilities=False,
        overwrite=True,
        num_processes_preprocessing=2,
        num_processes_segmentation_export=2,
    )
    print("✅ Done ->", OUT_DIR)
except Exception:
    traceback.print_exc()


Cases (CCTH-only): 7 / Total: 61
Sample: ['CCTH-A07_0000.nii.gz', 'CCTH-A08_0000.nii.gz', 'CCTH-B01_0000.nii.gz', 'CCTH-B02_0000.nii.gz', 'CCTH-B04_0000.nii.gz']
Init from: /workspace/nnunet_seg_model/models/Dataset505_PelvicBone/nnUNetTrainer__nnUNetPlans__3d_fullres
There are 7 cases in the source folder
I am processing 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 7 cases that I would like to predict
nnUNet_raw is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
nnUNet_results is not defined and nnU-Net 

 18%|█▊        | 14/80 [00:05<00:27,  2.43it/s]

old shape: (31, 612, 633), new_shape: [ 39 637 658], old_spacing: [np.float64(5.0), np.float64(0.40625), np.float64(0.40625)], new_spacing: [3.9999987840652467, 0.3905999958515167, 0.3905999958515167], fn_data: functools.partial(<function resample_data_or_seg_to_shape at 0x76f12edd6a70>, is_seg=False, order=3, order_z=0, force_separate_z=None)
None


 25%|██▌       | 20/80 [00:07<00:25,  2.36it/s]

old shape: (40, 638, 638), new_shape: [ 50 664 664], old_spacing: [np.float64(5.0), np.float64(0.40625), np.float64(0.40625)], new_spacing: [3.9999987840652467, 0.3905999958515167, 0.3905999958515167], fn_data: functools.partial(<function resample_data_or_seg_to_shape at 0x7d99b00daa70>, is_seg=False, order=3, order_z=0, force_separate_z=None)


...

Input shape: torch.Size([1, 46, 643, 664])
step_size: 0.5
mirror_axes: (0, 1, 2)
n_steps 80, image size is torch.Size([46, 643, 664]), tile_size [20, 320, 256], tile_step_size 0.5
steps:
[[0, 9, 17, 26], [0, 108, 215, 323], [0, 102, 204, 306, 408]]
move image to device cuda
preallocating results arrays on device cuda


100%|██████████| 80/80 [00:45<00:00,  1.76it/s]


Input shape: torch.Size([1, 46, 643, 664])
step_size: 0.5
mirror_axes: (0, 1, 2)
n_steps 80, image size is torch.Size([46, 643, 664]), tile_size [20, 320, 256], tile_step_size 0.5
steps:
[[0, 9, 17, 26], [0, 108, 215, 323], [0, 102, 204, 306, 408]]
move image to device cuda
preallocating results arrays on device cuda


100%|██████████| 80/80 [00:45<00:00,  1.76it/s]


Prediction done
sending off prediction to background worker for resampling and export
done with CCTH-B08
✅ Done -> /MRI/seg_out


In [12]:
import os
import shutil
import nibabel as nib
import numpy as np

# 입력: Stage 1 결과
input_dir = "/MRI/seg_out"
# 출력: 필터링된 결과
output_dir = "/workspace/MRI/seg_out"
os.makedirs(output_dir, exist_ok=True)

# threshold 조정 가능 (예: 0.06 이상일 경우만 채택)
confidence_threshold = 0.06

for fname in os.listdir(input_dir):
    if not fname.endswith(".nii.gz"):
        continue
    path = os.path.join(input_dir, fname)
    try:
        data = nib.load(path).get_fdata()
        confidence = np.mean(data == 1)

        if confidence > confidence_threshold:
            shutil.copy(path, os.path.join(output_dir, fname))
            print(f"✅ Kept {fname} (confidence: {confidence:.3f})")
        else:
            print(f"❌ Discarded {fname} (confidence: {confidence:.3f})")

    except Exception as e:
        print(f"❌ Failed to process {fname}: {e}")


❌ Discarded TCGA-VS-A8EB.nii.gz (confidence: 0.029)
❌ Discarded TCGA-VS-A8EC.nii.gz (confidence: 0.025)
❌ Discarded TCGA-VS-A8EG.nii.gz (confidence: 0.044)
❌ Discarded TCGA-VS-A8EH.nii.gz (confidence: 0.051)
❌ Discarded TCGA-VS-A8EI.nii.gz (confidence: 0.030)
❌ Discarded TCGA-VS-A8EJ.nii.gz (confidence: 0.027)
❌ Discarded TCGA-VS-A8EK.nii.gz (confidence: 0.041)
❌ Discarded TCGA-VS-A8EL.nii.gz (confidence: 0.036)
❌ Discarded TCGA-VS-A8Q8.nii.gz (confidence: 0.030)
❌ Discarded TCGA-VS-A8Q9.nii.gz (confidence: 0.047)
❌ Discarded TCGA-VS-A8QA.nii.gz (confidence: 0.056)
❌ Discarded TCGA-VS-A8QC.nii.gz (confidence: 0.034)
❌ Discarded TCGA-VS-A8QF.nii.gz (confidence: 0.031)
❌ Discarded TCGA-VS-A8QH.nii.gz (confidence: 0.045)
❌ Discarded TCGA-VS-A8QM.nii.gz (confidence: 0.044)
❌ Discarded TCGA-VS-A94W.nii.gz (confidence: 0.035)
❌ Discarded TCGA-VS-A94X.nii.gz (confidence: 0.040)
❌ Discarded TCGA-VS-A94Y.nii.gz (confidence: 0.049)
❌ Discarded TCGA-VS-A94Z.nii.gz (confidence: 0.053)
❌ Discarded 